---

## 📋 Agents Information

---

### Simple Prescription Agent
- **Type**: Simple Agent
- **Name**: `simple_prescription_agent`
- **Subagents**: None
- **Returned State**: `results_criticality`
- **Dashboard Metrics**: Overall prescription safety level (low/medium/high)

Schema Output Example:
```json
{
  "overall_criticality": "medium",
  "description": "Overall safety assessment of the prescription."
}
```

### Parallel Analysis Agent
- **Type**: Parallel Agent
- **Name**: `parallel_analyzer_agent`
- **Subagents**: `drug_analysis_agent`, `dose_drug_analysis_agent`, `route_drug_analysis_agent`
- **Returned State**: `synthesized_results_criticality`
- **Dashboard Metrics**: Individual criticality levels for drug, dose, and route analysis

Schema Output Example:
```json
{
  "level_drug": "medium",
  "level_dose": "high", 
  "level_route": "low",
  "description": "Synthesis of drug, dose, and route analysis criticality levels."
}
```

### Sequential Health Analyzer Agent
- **Type**: Sequential Agent
- **Name**: `sequential_analyzer_agent`
- **Subagents**: `general_health_agent`, `treatment_assessment_agent`, `synthesizer_health_report_agent`
- **Returned State**: `synthesized_health_report`
- **Dashboard Metrics**: 
  - Treatment Duration Criticality (low/medium/high)
  - Patient Compliance Risk (low/medium/high) 
  - Lifestyle Impact Level (low/medium/high)
  - Monitoring Frequency Criticality (low/medium/high)
  - Executive Summary (text)
  - Actionable Recommendations (text)

Schema Output Example:
```json
{
  "treatment_duration_criticality": "medium",
  "patient_compliance_criticality": "high",
  "lifestyle_impact_criticality": "low", 
  "monitoring_frequency_criticality": "medium",
  "executive_summary": "Patient profile and treatment overview...",
  "actionable_recommendations": "Specific healthcare provider actions..."
}
```

---

## 📊 Dashboard Integration

**Agent Outputs → Dashboard Features:**

1. **Safety Alerts Panel**: Simple + Parallel agent criticality levels
2. **Treatment Timeline Widget**: Sequential agent duration assessment  
3. **Patient Risk Matrix**: Compliance + lifestyle impact scores
4. **Monitoring Schedule**: Frequency criticality + recommendations
5. **Executive Summary Card**: Consolidated patient overview
6. **Action Items List**: Prioritized healthcare provider tasks

**Main Function:** `run_agent(agent_name, input_data, user_id)`

**Workflow:** Create Session → Execute Agent → Get State → Delete Session

**API:** `http://localhost:8000` (ADK server running)

In [32]:
r = requests.get(f"{BASE_URL}/list-apps", timeout=30)
r.raise_for_status()
apps = r.json()
apps

['parallel_analyzer_agent',
 'search_agent',
 'sequential_analyzer_agent',
 'simple_prescription_agent']

In [33]:
import requests
import json
import uuid

# Configuração
BASE_URL = "http://localhost:8000"
BASE_URL = "https://adk-server-health-894271896157.europe-west1.run.app"

# Nomes dos agentes disponíveis
AGENTS = {
    "lead": "lead_qualification_agent",
    "search": "search_agent", 
    "essay": "essay_analyzer_agent",
    "prescription": "simple_prescription_agent"
}

def get_all_apps():
    """
    Recupera a lista de todos os aplicativos disponíveis na API ADK.
    
    Returns:
        list - Lista de aplicativos.
    """
    response = requests.get(f"{BASE_URL}/list-apps")
    return response.json()

all_apps = get_all_apps()
all_apps

['parallel_analyzer_agent',
 'search_agent',
 'sequential_analyzer_agent',
 'simple_prescription_agent']

In [34]:
def run_agent(agent_name: str, input_data: str, user_id: str = "u_test") -> dict:
    """
    Executa um agente e retorna o estado completo.
    Cria e deleta a sessão automaticamente.
    """
    session_id = f"s_{uuid.uuid4().hex[:8]}"
    
    # 1. Criar sessão
    requests.post(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions/{session_id}",
        json={"state": {}, "user_id": user_id, "session_id": session_id}
    )
    
    # 2. Executar agente
    requests.post(
        f"{BASE_URL}/run",
        json={
            "appName": agent_name,
            "userId": user_id,
            "sessionId": session_id,
            "newMessage": {"parts": [{"text": input_data}], "role": "user"}
        }
    )
    
    # 3. Obter resultado
    response = requests.get(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions/{session_id}"
    )
    
    # 4. Deletar sessão
    requests.delete(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions/{session_id}"
    )
    
    return response.json().get("state", {})


def get_all_sessions(agent_name: str, user_id: str = "u_test") -> list:
    """Obtém todas as sessões de um usuário para um agente específico."""
    response = requests.get(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions"
    )
    return response.json()

In [35]:
import json

# load data
with open("../data/inputs_to_agent_fake_mimic3.json", "r", encoding="utf-8") as f:
    fake_admissions = json.load(f)
    
regular = fake_admissions[0]
poisoned = fake_admissions[9]

regular_example = f"""
data: {regular["admission_str"]}

current_prescription: {regular["current_prescription"]}
"""

poisoned_example = f"""
data: {poisoned["admission_str"]}
current_prescription: {poisoned["current_prescription"]}
"""

very_poisoned_example = f"""

data: Subject ID: 9
Admission ID: 199395
Admission Date: 6959286900000
Discharge Date: 6960348000000
Insurance: Medicare
Age: None
Gender: F
Language: None
Marital Status: MARRIED
Ethnicity: WHITE
Creatinine: 2.3 mg/dL
Hemoglobin: 11.0 g/dL
Potassium: 4.6 mEq/L
Sodium: 143.0 mEq/L
Notes (24h Summary): None
Prescriptions:   - Drug: Oxycodone-Acetaminophen, Type: MAIN, Dose: 1-2 TAB, Form: None, Route: PO, Start: 2190-07-16 00:00:00, Stop: 2190-07-18 00:00:00--------------------------------
  - Drug: Oxycodone-Acetaminophen, Type: MAIN, Dose: 5-10 ml, Form: None, Route: PO, Start: 2190-07-16 00:00:00, Stop: 2190-07-18 00:00:00--------------------------------
  - Drug: Amiodarone HCl, Type: MAIN, Dose: 150 mg, Form: None, Route: IV, Start: 2190-07-17 00:00:00, Stop: 2190-07-18 00:00:00--------------------------------
  - Drug: GlipiZIDE, Type: MAIN, Dose: 7.5 mg, Form: None, Route: PO, Start: 2190-07-17 00:00:00, Stop: 2190-07-21 00:00:00--------------------------------
  - Drug: Amiodarone HCl, Type: MAIN, Dose: 400 mg, Form: None, Route: PO, Start: 2190-07-18 00:00:00, Stop: 2190-07-21 00:00:00--------------------------------
  - Drug: Potassium Chloride, Type: MAIN, Dose: 20 mEq, Form: None, Route: PO, Start: 2190-07-18 00:00:00, Stop: 2190-07-19 00:00:00--------------------------------
  - Drug: Oxycodone-Acetaminophen, Type: MAIN, Dose: 1-2 TAB, Form: None, Route: PO, Start: 2190-07-19 00:00:00, Stop: 2190-07-21 00:00:00--------------------------------
  - Drug: Amiodarone HCl, Type: MAIN, Dose: 150 mg, Form: None, Route: IV, Start: 2190-07-19 00:00:00, Stop: 2190-07-20 00:00:00--------------------------------
  - Drug: Magnesium Sulfate, Type: MAIN, Dose: 2 gm, Form: None, Route: IV, Start: 2190-07-19 00:00:00, Stop: 2190-07-20 00:00:00--------------------------------
  - Drug: Metoprolol, Type: MAIN, Dose: 12.5 mg, Form: None, Route: PO, Start: 2190-07-20 00:00:00, Stop: 2190-07-21 00:00:00--------------------------------
----------------------------------------

current_prescription: 'drug': 'Fentanyl Patch', 'drug_type': 'MAIN', 'dose_val': '22500', 'dose_unit': 'PUFF', 'form': None, 'route': 'DWELL', 'starttime': '2190-07-20 00:00:00', 'stoptime': '2190-07-25 00:00:00'
"""

In [37]:
state_regular = run_agent("parallel_analyzer_agent", 
                  regular_example)

state_poisoned = run_agent("parallel_analyzer_agent", 
                  poisoned_example)

state_very_poisoned = run_agent("parallel_analyzer_agent", 
                  very_poisoned_example)

In [38]:
state_regular

{'state': {},
 'user_id': 'u_test',
 'session_id': 's_f554391f',
 'drug_analysis': 'MethylPREDNISolone Sodium Succinate does not have any critical safety alerts that warrant immediate intervention. Therefore, the safety profile is considered LOW.',
 'route_drug_analysis': 'The prescription specifies subcutaneous administration of MethylPREDNISolone Sodium Succinate. While IV administration is common, SC administration is less typical and might affect absorption rates, but does not present a high risk of immediate toxicity. GRADE: MEDIUM\n',
 'dose_drug_analysis': 'The prescribed dose of MethylPREDNISolone Sodium Succ (1500-2900 mg SC) is significantly above typical doses and may represent a critical overdose risk, especially given the subcutaneous route. GRADE: HIGH\n',
 'synthesized_results_criticality': {'level_drug': 'LOW',
  'level_dose': 'HIGH',
  'level_route': 'MEDIUM',
  'description': 'The synthesized safety assessment indicates a HIGH concern due to the significantly elevated

In [39]:
print(regular_example)


data: Subject ID: 0
Admission ID: 169938
Admission Date: 4323585540000
Discharge Date: 4324203900000
Insurance: Medicare
Age: None
Gender: M
Language: None
Marital Status: UNKNOWN (DEFAULT)
Ethnicity: WHITE
Creatinine: 1.1 mg/dL
Hemoglobin: 10.0 g/dL
Potassium: 3.7 mEq/L
Sodium: 142.0 mEq/L
Notes (24h Summary): None
Prescriptions:   - Drug: SW, Type: BASE, Dose: 100 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------
  - Drug: NS, Type: BASE, Dose: 500 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------
  - Drug: NS, Type: BASE, Dose: 500 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------
  - Drug: Iso-Osmotic Dextrose, Type: BASE, Dose: 200 ml, Form: None, Route: IV, Start: 2107-01-07 00:00:00, Stop: 2107-01-08 00:00:00--------------------------------
  - Drug: Fat Emulsion 20%, Type: BASE, Dose: 250 ml

In [21]:
state_regular_sequential

{'state': {},
 'user_id': 'u_test',
 'session_id': 's_a268b691',
 'general_health_report': "Here's a structured health report based on the provided patient data:\n\n**1. Patient Profile Summary**\n\n*   **Subject ID:** 0\n*   **Admission ID:** 169938\n*   **Gender:** Male (M)\n*   **Age:** Not specified\n*   **Ethnicity:** White\n*   **Marital Status:** Unknown\n*   **Insurance:** Medicare\n*   **Admission Date:** 4323585540000 (Timestamp format)\n*   **Discharge Date:** 4324203900000 (Timestamp format)\n\n**2. Identified Health Conditions**\n\n*   Anemia (Hemoglobin: 10.0 g/dL, which is below the normal range).\n*   Creatinine: 1.1 mg/dL - Within normal limits, suggesting adequate kidney function for the moment.\n*   Potassium: 3.7 mEq/L and Sodium: 142.0 mEq/L - Both within normal limits.\n\n**3. Analysis of Prescriptions in Relation to Clinical Picture**\n\n*   **Baseline IV Fluids (SW, NS, Iso-Osmotic Dextrose, D5W):** The patient received various intravenous fluids (Sterile Water,

In [23]:
state_very_poisoned_sequential

{'state': {},
 'user_id': 'u_test',
 'session_id': 's_1f23abda',
 'general_health_report': "## Patient Health Analysis Report\n\n**1. Patient Profile Summary:**\n\n*   **Subject ID:** 9\n*   **Admission ID:** 199395\n*   **Admission Date:** 6959286900000\n*   **Discharge Date:** 6960348000000\n*   **Age:** Not available\n*   **Gender:** Female\n*   **Marital Status:** Married\n*   **Ethnicity:** White\n*   **Insurance:** Medicare\n*   **Creatinine:** 2.3 mg/dL (Elevated)\n*   **Hemoglobin:** 11.0 g/dL (Low)\n*   **Potassium:** 4.6 mEq/L (Normal)\n*   **Sodium:** 143.0 mEq/L (Normal)\n\n**2. Identified Health Conditions:**\n\nBased on the available data, the following potential health conditions can be inferred:\n\n*   **Renal Impairment:** Elevated creatinine levels (2.3 mg/dL) suggest possible kidney dysfunction.\n*   **Anemia:** Low hemoglobin levels (11.0 g/dL) indicate anemia.\n*   **Pain Management:** The prescription of Oxycodone-Acetaminophen suggests the patient is experiencing